# Boids

This is a simple 2DPGA implementation of boids algorithm. I got inspired to try this after watching this Smarter Every Day video:

[<img src=https://img.youtube.com/vi/4LWmRuB-uNU/maxresdefault.jpg width=360/>](https://www.youtube.com/watch?v=4LWmRuB-uNU)

Further inspiration for the implementation was taken from [the code base shown in the video](https://eater.net/boids).

The current implementation just copies the algorithm as is; in the future I would like to figure out how to do this using forgues and proper PGA based dynamics. However, this notebook nicely **demonstrates the kingdon+numpy array interoperability**.

In [1]:
%pip install -q kingdon anywidget==0.9.9 ipywidgets==8.1.3

Note: you may need to restart the kernel to use updated packages.


In [2]:
from kingdon import Algebra
import numpy as np

In [3]:
d = 2
alg = Algebra(d, 0, 1, graded=False)

In [4]:
turn_factor = 0.1
visual_range = 20
protected_range = 8
centering_factor = 0.005
avoid_factor = 0.005
matching_factor = 0.02
max_speed_of_boid = 2
min_speed_of_boid = 0.5
box_size = 100
nboids = 30

boidvals = np.ones((d+1, nboids))
boidvals[1:] = box_size * np.random.uniform(-0.5, 0.5, (d, nboids))
boidvelvals = np.zeros((d+1, nboids))
boidvelvals[1:] = max_speed_of_boid * np.random.uniform(-0.5, 0.5, (d, nboids)) 
boids = alg.vector(boidvals).dual()
boidvels = alg.vector(boidvelvals).dual()  #, keys=('e1', 'e2')

# Create a simple shape for the boids.
boidshape = [alg.vector(e0=1, e1=0, e2=2).dual(), alg.vector(e0=1, e1=0.5, e2=-1).dual(), alg.vector(e0=1, e1=0, e2=0).dual(), alg.vector(e0=1, e1=-0.5, e2=-1).dual()]

The next cell makes the boundaries for the boids.

In [5]:
points = [
    alg.vector([1.0, *((float(x)-0.5)*box_size for x in bin(i)[2:].zfill(d))]).dual()
    for i in range(2**d)
]
edges = [
    [a, b]
    for i, a in enumerate(points)
    for j, b in enumerate(points)
    if not (i<=j or (i ^ j) & ((i ^ j) - 1))
]
boundaries = [alg.rp(*edges[0]), -alg.rp(*edges[1]), alg.rp(*edges[2]), -alg.rp(*edges[3])] # Handpicked signs such that the signs are always positive if a boid is on the inside.
boundary_direction = [(b.normalized() @ alg.blades.e0.dual()).dual() for b in boundaries]

In [6]:
def graph_func():
    global boids, boidvels
    
    for i, (boid, boidvel) in enumerate(zip(boids, boidvels)):
        d = (boid & boids).norm().e  # Distance to every other boid.
        neighbors = d < visual_range
        pos_avg = boids[neighbors].map(np.mean)
        vel_avg = boidvels[neighbors].map(np.mean)
        # Strive to stay together
        boidvels[i] += (pos_avg - boid) * centering_factor
        # Strive to match velocity
        boidvels[i] += (vel_avg - boidvel) * matching_factor
        # Avoid collision with those neighbors too close
        too_close = d < protected_range
        close = (boid - boids[too_close]).map(np.sum)
        boidvels[i] += close*avoid_factor
        
    # Reflect of the edges.
    
    boundary_signs = boids ^ boundaries
    for signed_distance, direction in zip(boundary_signs, boundary_direction):
        # if the sign of pss is negative, that boid is out of bounds! Make it fly back in the opposite direction.
        out_of_bounds = signed_distance.e012 < 0
        boidvels[out_of_bounds] += turn_factor * direction#* (direction - boids[out_of_bounds])# * signed_distance.e012[out_of_bounds]

    # Impose the speed of boid
    B = boidvels.dual().map(np.nan_to_num)
    speed = B.norm().e
    maniacs = speed > max_speed_of_boid
    losers = speed < min_speed_of_boid
    B[maniacs] = B[maniacs].normalized() * max_speed_of_boid
    B[losers] = B[losers].normalized() * min_speed_of_boid
    boidvels = B.undual()
    
    boids = boids + boidvels
    return [
        *[(-boid*alg.blades.e12).sqrt() >> ((v.dual()*alg.blades.e2).sqrt() >> boidshape) for boid, v in zip(boids, boidvels)],
    ]



In [7]:
alg.graph(
    graph_func,
    animate=1,
    scale=0.03,
    lineWidth=6,
    labels=True,
    width='100%',
)

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0', 'e01', 'e02', '0', '0', …